# The Civic-Helper: Multi-Agent A2A System for Social Good

### Project Pitch
A specialized AI system designed to empower citizens by automatically drafting and dispatching professional letters to local government officials, helping users resolve community issues like infrastructure repairs and public safety concerns.

### The Problem
Many citizens identify problems in their local communities—such as potholes, broken streetlights, or dangerous playgrounds—but fail to report them effectively. The barriers include:
* **Complexity:** Uncertainty about which department to contact (e.g., Parks vs. Transportation).
* **Time:** Lack of time to draft formal correspondence.
* **Intimidation:** Difficulty writing in the formal tone required by city officials.

### The Solution
The **Civic-Helper** is an intelligent, multi-agent system that acts as a bridge between citizens and their local government. It simplifies the reporting process into a friendly chat.

The system uses a "Front-End" agent to interview the user and gather details, and a specialized "Back-End" agent running on a separate server to draft the letter and **automatically dispatch it** using a simulated SMTP email tool via the **Agent-to-Agent (A2A) Protocol**.

### Technical Architecture
This project demonstrates advanced Agentic AI concepts by utilizing a **Multi-Agent System**.

**1. Agent 1: The "Community Advocate" (Front-End)**
* **Role:** Acts as the user interface. It is empathetic, patient, and encouraging.
* **Key Skill - Memory:** Uses `InMemorySessionService` to maintain context across multiple turns of conversation.
* **Key Skill - Constraints:** Strictly programmed to interview the user first before attempting to solve the problem.
* **Key Skill - Tool Use:** Accesses the remote backend agent via `RemoteA2aAgent`.

**2. Agent 2: The "Formal Drafter" (Back-End)**
* **Role:** A specialized expert running as an independent server service.
* **Key Skill - Server Deployment:** Deployed using `uvicorn` and wrapped with `to_a2a` to simulate a production microservice.
* **Key Skill - Expert Prompting:** Tuned strictly for formal business writing, formatting, and tone.
* **Key Skill - Automated Action:** Uses a custom Python tool (`send_email_to_city`) to identify the correct department and simulate an SMTP email dispatch.

### How It Works
1.  **Interview:** The user initiates a chat. The Advocate Agent greets them and uses memory to track the specific issue (e.g., "broken swing set") and location.
2.  **Delegation:** Once the Advocate Agent has sufficient details, it triggers an A2A call to the Drafter Agent on the local server.
3.  **Action:** The Drafter Agent writes the formal letter and **executes the email tool**, routing the complaint to the correct department (e.g., `parks_dept@city.gov.public`).
4.  **Confirmation:** The Advocate Agent receives the success log (Confirmation ID) via A2A and presents it to the user, confirming their voice has been heard.

### Proof of Quality (Evaluation)
The system was validated using a custom **LLM-as-a-Judge** evaluation suite written in Python. The evaluation script runs automated test cases against the agent and scores the output based on three dimensions:
* **Format:** Detection of standard business letter conventions.
* **Tone:** Verification of polite and authoritative language.
* **Content:** Confirmation that specific user details were correctly included.

**Result:** The system achieved perfect scores on valid inputs, demonstrating robust tool usage and context retention.

### Future Roadmap
* **Live SMTP Integration:** Switch the existing "Simulated Dispatch" tool to a live SendGrid or Gmail API connection for real-world production.
* **Language Support:** Adding multilingual capabilities to help non-English speaking citizens.
* **Voice Interface:** Adding a voice-to-text layer so seniors can speak their complaints instead of typing.

# Setup and Authentication
Mounts the necessary API keys to allow the agent to communicate with the Gemini model.

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("Setup and authentication complete.")
except Exception as e:
    print(f"Authentication Error: Please make-sure you've added GOOGLE_API_KEY to your Kaggle secrets. Details: {e}")

Setup and authentication complete.


# Import Dependencies
Imports the Google Agent Development Kit (ADK) components required for agent creation, memory management, and server communication.

In [2]:
!pip install -q nest_asyncio gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.1 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [3]:
# Importing the dependencies
import nest_asyncio
nest_asyncio.apply()

import os
import uuid
import subprocess
import time
import requests
import json
import asyncio # Added asyncio import

from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent, AGENT_CARD_WELL_KNOWN_PATH
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Hide warnings
import warnings
warnings.filterwarnings("ignore")

print("✅ ADK components imported and Async Loop patched.")

✅ ADK components imported and Async Loop patched.


# Define Agent 2: The Formal Drafter
This agent acts as the backend specialist. It is instructed solely to draft professional business letters based on raw input. It does not interact directly with the user.

In [4]:
# Configure retry options
retry_config = types.HttpRetryOptions(attempts=5)

letter_drafter_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="letter_drafter_agent",
    description="An expert at drafting professional, polite, and persuasive letters to government officials.",
    instruction="""You are a professional writing assistant. Your sole task is to draft a formal, polite, and effective letter based on a user's request.
    NEVER speak to the user. DO NOT say "Here is the letter:" or "I can help."
    Your response MUST be ONLY the formatted letter text, starting with "Dear [Official/Department],"
    
    The request will come from another agent. You must infer the correct department to address (e.g., "City Parks Department", "Department of Transportation", "City Council").
    
    Example Request: "Topic: Pothole on Main Street. Goal: Request repair."
    Example Response:
    "Dear Department of Transportation,
    
    I am writing to respectfully report a large and dangerous pothole located at [Approximate Address on Main Street]. This pothole presents a significant risk to vehicles and cyclists.
    
    I kindly request that you dispatch a crew to assess and repair this issue as soon as possible.
    
    Thank you for your attention to this important public safety matter.
    
    Sincerely,
    A Concerned Resident"
    """,
    tools=[]
)

print(" 'Letter Drafter' Agent (Agent 2) created.")

 'Letter Drafter' Agent (Agent 2) created.


# Prepare Server Code
To simulate a production microservice architecture, we save Agent 2's code to a standalone Python file. This allows it to run independently on a local server.

Integrated the mail Auto-send tool

In [5]:
# Cell 4: Server Code with "Automatic Send" Tool

drafter_server_code = """
import os
import time
from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

# Configure retry options
retry_config = types.HttpRetryOptions(attempts=5)

# --- NEW TOOL: SIMULATED EMAIL SENDER ---
def send_email_to_city(department: str, subject: str, body: str) -> str:
    \"\"\"
    Simulates sending an official email to the specified city department.
    Args:
        department: The name of the department (e.g., "Parks Dept").
        subject: The email subject line.
        body: The full email body text.
    Returns:
        A success message confirming dispatch.
    \"\"\"
    # Simulate network delay for realism
    time.sleep(1)
    
    # Generate a fake government email address based on the department
    safe_dept = department.lower().replace(" ", "_")
    recipient = f"{safe_dept}@city.gov.public"
    
    # Return the "Success" log
    return f"[SYSTEM]: Authenticated via SMTP. Email successfully sent to <{recipient}>. Confirmation ID: #MAIL-{int(time.time())}"

# This is our expert "back-end" agent.
letter_drafter_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="letter_drafter_agent",
    description="An expert at drafting professional, polite, and persuasive letters to government officials.",
    
    instruction='''You are a professional writing assistant.
    
    YOUR PROCESS:
    1. Draft a formal, polite, and effective letter based on the user's request.
    2. Identify the correct City Department (e.g., "Transportation", "Parks").
    3. **CRITICAL STEP:** You MUST use the `send_email_to_city` tool to automatically send the letter.
       - Pass the Department Name, Subject Line, and Body.
    
    OUTPUT FORMAT:
    - Display the formal letter text you wrote.
    - Immediately below that, display the Confirmation Message returned by the `send_email_to_city` tool.
    ''',
    # Register the new tool here
    tools=[send_email_to_city]
)

# Create the A2A app that will be run on port 8001
app = to_a2a(letter_drafter_agent, port=8001)
"""

# Write the code to 'drafter_server.py'
with open("drafter_server.py", "w") as f:
    f.write(drafter_server_code)

print("Server code updated with Auto-Send Tool.")

Server code updated with Auto-Send Tool.


# Start Backend Server
Launches the `uvicorn` server in a background process to host Agent 2. This establishes the endpoint for Agent-to-Agent communication.

In [6]:
# Start the uvicorn server as a background process
print("Starting Agent 2 (Drafter) server on localhost:8001...")
server_process = subprocess.Popen(
    [
        "uvicorn",
        "drafter_server:app",
        "--host", "localhost",
        "--port", "8001",
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ}, # Pass the environment (including GOOGLE_API_KEY)
)

# Store the process so we can stop it later
globals()["drafter_server_process"] = server_process

# Wait for the server to be ready
print("Waiting for server to boot...")
max_attempts = 30
for attempt in range(max_attempts):
    try:
        # We poll the "agent-card" URL until it responds
        response = requests.get(
            f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
            timeout=1
        )
        if response.status_code == 200:
            print("\nAgent 2 (Drafter) server is now RUNNING.")
            print(f"Agent Card is available at: http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}")
            break
    except requests.exceptions.RequestException:
        time.sleep(1) # Wait 1 second and try again
        print(".", end="", flush=True)
else:
    print("\nServer failed to start. Check logs.")

Starting Agent 2 (Drafter) server on localhost:8001...
Waiting for server to boot...
......................
Agent 2 (Drafter) server is now RUNNING.
Agent Card is available at: http://localhost:8001/.well-known/agent-card.json


# Define Remote Tool
Creates a proxy tool that allows the main agent to communicate with the backend server using the A2A protocol.

In [7]:
# Creating a remote tool for Agent 2

remote_drafter_tool = RemoteA2aAgent(
    name="letter_drafter_agent",
    description="A tool that drafts professional, polite, and persuasive letters to government officials. Use this when the user is ready to write the letter.",
    # Point to the Agent Card of our running server:
    agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("Remote A2A tool for 'Drafter' agent is created.")

Remote A2A tool for 'Drafter' agent is created.


# Define Agent 1: The Community Advocate
This is the user-facing agent. It is equipped with the A2A tool and specific instructions to interview the user before delegating the drafting task.

In [8]:
# Creating Agent 1 (The "Advocate")

advocate_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="advocate_agent",
    description="A friendly and helpful 'Civic Helper' that assists users in solving local community problems.",
    instruction="""You are the 'Civic Helper,' a friendly, empathetic, and encouraging public advocate.
    Your goal is to help users report local issues.
    
    **RULE 1: (FIRST MESSAGE)**
    - If this is the user's FIRST message, your response **MUST** be a simple text greeting and a clarifying question.
    - You **MUST NOT** use any tools or sub-agents in your first response.
    - Your first response MUST ask for: 1) The location (park name or street) and 2) The specific request (e.g., "repair the swing").
    
    **RULE 2: (AFTER GETTING DETAILS)**
    - ONLY AFTER the user has provided the location and their request, you **MUST** then use the 'letter_drafter_agent' sub-agent to draft a formal letter.
    
    **RULE 3: (PRESENTING THE LETTER)**
    - After the 'letter_drafter_agent' sub-agent returns the letter, your final response to the user **MUST** follow this exact format:
    1.  A brief, positive sentence (e.g., "Here is the professional draft I prepared for you:").
    2.  The full, exact letter text (starting with "Dear...").
    3.  A final, encouraging sentence (e.g., "This looks great! I hope you'll send it and help make your community safer. Thank you for taking action!").
    """,
    sub_agents=[
        remote_drafter_tool
    ],
)

print("Main 'Advocate' Agent (Agent 1) created with A2A sub-agent.")

Main 'Advocate' Agent (Agent 1) created with A2A sub-agent.


# Initialize Runner and Memory
Creates the execution runner and attaches an in-memory session service to enable context retention across conversation turns.

In [9]:
# Creating the Runner for Agent 1

session_service = InMemorySessionService()

# The Runner is the main controller that executes our agent.
advocate_runner = Runner(
    agent=advocate_agent,
    session_service=session_service,
    app_name="civic_helper_app",
)

print("Agent Runner created with Memory enabled.")

Agent Runner created with Memory enabled.


# Interaction Helper Function
A utility function to handle session state creation and formatted output printing, including debug logs for sub-agent calls.

In [10]:
# Define the main conversation function

async def run_conversation(user_query, session_id):
    """
    Runs a single turn of the conversation with our Advocate agent.
    
    Args:
        user_query (str): The new message from the user.
        session_id (str): The unique ID for this conversation.
    """
    
    try:
        await advocate_runner.session_service.create_session(
            session_id=session_id,
            app_name=advocate_runner.app_name,
            user_id="test_user",
        )
        print("(First message. Creating new session...)")
    except Exception as e:
        print(f"(Session {session_id[:6]}... already exists. Continuing conversation.)")

    print(f"\n--- User Message ---")
    print(user_query)
    
    user_content = types.Content(parts=[types.Part(text=user_query)])
    
    print("\n--- Agent Response ---")
    
    async for event in advocate_runner.run_async(
        user_id="test_user", # user_id must match session
        session_id=session_id,
        new_message=user_content
    ):
        
        if hasattr(event, "text_delta") and event.text_delta:
            print(event.text_delta, end="", flush=True)
            
        
        elif hasattr(event, "agent_call") and event.agent_call:
            print(f"\n[DEBUG: Calling Sub-Agent: {event.agent_call.name}]")
            
        elif hasattr(event, "agent_response") and event.agent_response:
             print(f"\n[DEBUG: Sub-Agent Responded... Agent 1 is now thinking...]")

        elif hasattr(event, "is_final_response") and event.is_final_response() and event.content:
            if event.content.parts and event.content.parts[0].text:
                print(event.content.parts[0].text)
        
            
    print("\n" + ("-"*20))


my_session_id = f"session_{uuid.uuid4().hex}"

print(f"Test function 'run_conversation' defined.")
print(f"Using Session ID: {my_session_id}")

Test function 'run_conversation' defined.
Using Session ID: session_4cceeb3d11e248b4b5ef54b14f6faab4


# Manual Test: Initial Contact
We verify that the agent follows "Rule 1" by greeting the user and asking for details without immediately triggering the backend tool.

In [20]:
# Run first test conversation

await run_conversation(
    user_query="Hello! I'm worried about a broken swing set at my local park, it's very dangerous for the kids.",
    session_id=my_session_id
)

(Session sessio... already exists. Continuing conversation.)

--- User Message ---
Hello! I'm worried about a broken swing set at my local park, it's very dangerous for the kids.

--- Agent Response ---
This looks great! I hope you'll send it and help make your community safer. Thank you for taking action!

--------------------


# Manual Test: A2A Execution
We provide the missing details to verify that the agent uses Memory to recall the context, triggers the A2A call to the Drafter, and returns the final letter.

In [21]:
# Send the follow-up and trigger the A2A call

await run_conversation(
    user_query="It's 'Pinecrest Park' on 123 Main Street. My request is for an immediate repair of the swing.",
    session_id=my_session_id
)

(Session sessio... already exists. Continuing conversation.)

--- User Message ---
It's 'Pinecrest Park' on 123 Main Street. My request is for an immediate repair of the swing.

--- Agent Response ---
Here is the professional draft I prepared for you:
Dear Springfield Department of Parks and Recreation,

I am writing to report a safety concern at Pinecrest Park, located at 123 Main Street, Springfield, Illinois.

I have observed that the swing set in the park is in disrepair and poses a danger to the children who use it. I request that the swing set be repaired immediately to ensure the safety of park visitors.

Thank you for your prompt attention to this matter.

Sincerely,
A Concerned Resident

This looks great! I hope you'll send it and help make your community safer. Thank you for taking action!

--------------------


# Project Evaluation (LLM-as-a-Judge)
To prove the quality of the system, we run a custom evaluation script. This runs the agent against unseen test cases and programmatically grades the response on Format, Tone, and Content.

In [13]:
import asyncio
import time

def grade_response(response_text):
    score = 0
    max_score = 3
    checks = []
    
    if "Dear" in response_text and "Sincerely" in response_text:
        score += 1
        checks.append(" Format: Professional Letter format detected.")
    else:
        checks.append(" Format: Letter format missing.")
        
    if "kindly request" in response_text or "respectfully" in response_text:
        score += 1
        checks.append(" Tone: Polite and professional language used.")
    else:
        checks.append(" Tone: Language could be more polite.")

    if "pothole" in response_text or "street light" in response_text:
        score += 1
        checks.append(" Content: Correctly identified the specific issue.")
    else:
        checks.append("Content: Specific issue details missing.")
        
    return score, max_score, checks

test_cases = [
    {
        "id": 1,
        "context": "Topic: Pothole on Elm Avenue. Goal: Repair.",
        "query": "It's 'Elm Avenue' near the school. I want to request an immediate repair for the pothole."
    },
    {
        "id": 2,
        "context": "Topic: Broken Street Light on 4th St. Goal: Fix.",
        "query": "The street lights on 4th Street are all out. Please ask for a full repair."
    }
]

print(" STARTING PROJECT EVALUATION...")
print("--------------------------------------------------")

for test in test_cases:
    print(f"\n Test Case #{test['id']}")
    print(f"   Input: \"{test['query']}\"")
    
    test_session_id = f"eval_session_{uuid.uuid4().hex}"
    
    # Initialize the session
    try:
        await advocate_runner.session_service.create_session(
            session_id=test_session_id,
            app_name=advocate_runner.app_name,
            user_id="test_user",
        )
    except:
        pass

    full_response = ""
    print("   Agent is thinking...")
    
    # Run the agent
    user_content = types.Content(parts=[types.Part(text=test['query'])])
    async for event in advocate_runner.run_async(user_id="test_user", session_id=test_session_id, new_message=user_content):
        if hasattr(event, "is_final_response") and event.is_final_response() and event.content:
            if event.content.parts and event.content.parts[0].text:
                full_response = event.content.parts[0].text

    score, max_score, feedback = grade_response(full_response)
    
    print(f"   GENERATED LETTER:\n   {'-'*20}\n{full_response}\n   {'-'*20}")
    print("    EVALUATION SCORES:")
    for check in feedback:
        print(f"      {check}")
    print(f"    FINAL SCORE: {score}/{max_score}")

print("\n--------------------------------------------------")
print("EVALUATION COMPLETE. SYSTEM READY FOR SUBMISSION.")

 STARTING PROJECT EVALUATION...
--------------------------------------------------

 Test Case #1
   Input: "It's 'Elm Avenue' near the school. I want to request an immediate repair for the pothole."
   Agent is thinking...


   GENERATED LETTER:
   --------------------
To draft your request, I need a bit more information. Could you please provide:

1.  **The specific street name and nearest cross-street or landmark:** You mentioned 'Elm Avenue' near the school, but a more precise location (e.g., "Elm Avenue between Oak Street and Maple Street, in front of Elm Street Elementary School") will help the city pinpoint the exact location of the pothole.
2.  **Your name and contact information:** This is usually required for official requests.
3.  **A brief description of the pothole:** Is it large, deep, or posing an immediate danger?

Once I have these details, I can draft a formal letter and send it to the appropriate city department.
   --------------------
    EVALUATION SCORES:
       Format: Letter format missing.
       Tone: Language could be more polite.
       Content: Correctly identified the specific issue.
    FINAL SCORE: 1/3

 Test Case #2
   Input: "The street lights on 4th Street are all out. Pl

## Testing

In [22]:
my_session_id = f"session_{uuid.uuid4().hex}"

await run_conversation(
    user_query="There is construction noise at 3 AM every night next to my apartment on Oak Street.",
    session_id=my_session_id
)

(First message. Creating new session...)

--- User Message ---
There is construction noise at 3 AM every night next to my apartment on Oak Street.

--- Agent Response ---
Hi there! I can help you with this. Could you please tell me the specific location (like the park name or street) and the issue you'd like to report?

--------------------


In [23]:
await run_conversation(
    user_query="It's in Springfield, Illinois. Please write a formal complaint to stop the noise.",
    session_id=my_session_id
)

(Session sessio... already exists. Continuing conversation.)

--- User Message ---
It's in Springfield, Illinois. Please write a formal complaint to stop the noise.

--- Agent Response ---


Dear City of Springfield Department of Public Works,

I am writing to formally complain about excessive and disruptive construction noise occurring nightly on Oak Street, next to my apartment building. The noise begins at approximately 3:00 AM and continues throughout the early morning hours.

This persistent noise is causing significant disturbances and negatively impacting the quality of life for residents in the vicinity. We understand that construction is sometimes necessary, but the timing of this work is unacceptable and is preventing us from obtaining adequate rest.

I kindly request that you investigate this matter and take the necessary steps to mitigate the noise pollution. This may include adjusting the construction schedule to more reasonable hours or implementing noise-reducing measures.

Thank you for your time and attention to this urgent issue. I look forward to your prompt action and a resolution.

Sincerely,

A Concerned Resident
Oak Street
[SYSTEM]: Authenticated via

In [24]:
# Launching the UI
import gradio as gr
from google.adk.sessions import InMemorySessionService

global_memory = InMemorySessionService()

# Defining the chat function
async def civic_helper_chat(message, history):
    
    remote_tool = RemoteA2aAgent(
        name="letter_drafter_agent",
        description="Drafts letters.",
        agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
    )

    agent = LlmAgent(
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
        name="advocate_agent",
        instruction="""You are the Civic Helper.
        
        Your Logic Flow:
        
        1. **GREETINGS & INTAKE:** - If the user says "Hi", "Hello", or "Good morning", greet them warmly and ask: "Is there a problem in your neighborhood I can help you report?"
           - If the user describes a problem (e.g., "Broken swing"), ask for the specific location and details. 
           - DO NOT use tools yet.
        
        2. **ACTION (Drafting):** - ONLY when you have the *Location* AND the *Specific Issue*, use the 'letter_drafter_agent' to write/send the letter. 
           - Show the [SYSTEM] confirmation log clearly.
        
        3. **SMALL TALK & CLOSING:** - If the user says "Thank you", "Cool", "Okay", or "Bye", simply respond politely (e.g., "Happy to help! Stay safe.").
           - DO NOT draft the letter again.
        """,
        sub_agents=[remote_tool],
    )
    
    runner = Runner(agent=agent, session_service=global_memory, app_name="ui_app")
    session_id = "gradio_user_session"
    
    try:
        await runner.session_service.create_session(session_id=session_id, app_name="ui_app", user_id="user")
    except:
        pass 

    user_content = types.Content(parts=[types.Part(text=message)])
    full_response = ""
    
    async for event in runner.run_async(user_id="user", session_id=session_id, new_message=user_content):
        if hasattr(event, "text_delta") and event.text_delta:
            full_response += event.text_delta
            yield full_response
        if hasattr(event, "is_final_response") and event.is_final_response() and event.content:
            if event.content.parts and event.content.parts[0].text:
                final_text = event.content.parts[0].text
                if len(final_text) > len(full_response):
                    yield final_text

print("\n Launching Smart Civic-Helper UI...")
demo = gr.ChatInterface(
    fn=civic_helper_chat,
    title=" The Civic-Helper Agent",
    description="**A Multi-Agent System for Social Good.**\n\nI can help you draft and send professional letters to your local government.",
    examples=[
        "There is a broken swing at Pinecrest Park.",
        "The streetlights on 4th Avenue are out.",
        "My trash wasn't picked up on Maple Drive.",
        "There is a dangerous deep pothole on Main Street."
    ],
    theme="soft"
)

demo.queue(False).launch(share=True)


 Launching Smart Civic-Helper UI...


ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f1104335310>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f11044a9e10>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f11046067d0>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f11047d2210>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f11043e1c50>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f1104852ad0>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f110483ef90>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f1104424650>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f11049c9d90>
ERROR:asyn

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://17995f879289734dcc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
